# For Comprehensions

Indy Scala - November 2016


Brad Fritz | @bfritz | brad@fewerhassles.com

[List comprehensions](https://en.wikipedia.org/wiki/List_comprehension) (math!) ...

<img alt="https://en.wikipedia.org/wiki/List_comprehension" src="https://wikimedia.org/api/rest_v1/media/math/render/svg/8cc639ecfd1200a7fa8c18ed8624443b6f550ace"/>

Meet Scala for loops ...

```
val grid = for {
  row <- 1 to 2
  col <- 1 to 3
} yield (row, col)
```

## Examples...

In [ ]:
val grid = for {
    row <- 1 to 2
    col <- 1 to 3
} yield (row, col)

In [ ]:
for (row <- 1 to 2; col <- 1 to 3) println((row, col))

In [ ]:
val squares = for {
    x <- 1 to 10
    y <- 1 to 10
    sq = x * y if x == y
} yield sq

In [ ]:
val nestedOptions = for {
    o1 <- Some("string")
    o2 <- None
    o3 <- Some(123)
    _ = println("here!")
    o4 <- Some("other string")
} yield (s"$o1 $o2 $o3 $o4")

val output = nestedOptions.getOrElse("one or more values missing")

In [ ]:
case class Error()
case class UserInput()
case class Location()
case class Forecast()

def checkRateLimit(input: UserInput): Either[Error,Unit] = Right(Unit)
def geolocate(input: UserInput): Either[Error,Location] = Right(Location())
def findForecast(loc: Location): Either[Error,Forecast] = Right(Forecast())

val input = UserInput()
val forecast: Either[Error,Forecast] = for {
    rateLimited  <- checkRateLimit(input)
    userLocation <- geolocate(input)
    forecast     <- findForecast(userLocation)
} yield (forecast)

![Erik Bakker Play HttpResult example](Erik_Bakker_play_http_result_example.png)

[https://youtu.be/hGMndafDcc8?t=2130](https://youtu.be/hGMndafDcc8?t=2130)

Where:

```
type HttpResult[A] = EitherT[Future, Result, A]
```

## Resources

* [Journey to the Heart of the For-Yield](https://youtu.be/MHw-dDxC8Z4)
  <br/>Kelsey Gilmore-Innis at Pacific Northwest Scala 2013
* [Lovely for comprehensions: Scala's most useful syntactic tool](https://youtu.be/n_j2hzHQlNI)
  <br/>Adam Rosien at Scala IO 2014
* [Options in Futures, how to unsuck them](https://youtu.be/hGMndafDcc8)
  <br/>Eric Bakker at ScalaDays 2015 in Amsterdam
